In [ ]:
# =============================================================================
# Cell 1: Data Construction, Visualization & Descriptive Stats (FIXED)
# =============================================================================
!pip install praat-parselmouth pandas numpy matplotlib seaborn tqdm requests rpy2

# 👇 これが R (%%R) を使うために必須のコマンドです
%load_ext rpy2.ipython

import os
import glob
import re
import string
import requests
import zipfile
import io
import numpy as np
import pandas as pd
import parselmouth
from parselmouth.praat import call
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# 1. Setup & Directories (Robust Mount)
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive')
    except:
        print("⚠️ Initial mount failed. Retrying with force_remount...")
        drive.mount('/content/drive', force_remount=True)

BASE_DIR = "/content/drive/MyDrive/buckeye_self_repair"
RAW_DIR  = os.path.join(BASE_DIR, "raw")
OUT_DIR  = os.path.join(BASE_DIR, "output")
for d in [RAW_DIR, OUT_DIR]: os.makedirs(d, exist_ok=True)

# 2. Helper Functions
def normalize_token(token):
    if not token: return ""
    if token.startswith('<') or token.startswith('{'): return ""
    return token.strip(string.punctuation).lower()

def load_words_robust(filepath):
    """Buckeye .words loader: Uses next onset for duration calculation."""
    if not os.path.exists(filepath): return pd.DataFrame()
    data = []
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3 and not line.startswith(('#', '{')):
                try:
                    onset = float(parts[0])
                    raw_word = parts[2]
                    norm_w = normalize_token(raw_word)
                    if norm_w: data.append({"onset": onset, "word": norm_w})
                except ValueError: continue
    df = pd.DataFrame(data)
    if df.empty: return df
    df['offset'] = df['onset'].shift(-1)
    df = df.dropna(subset=['offset'])
    df['duration'] = df['offset'] - df['onset']
    df = df[df['duration'] > 0]
    df['nth'] = df.groupby('word').cumcount() + 1
    return df

def get_acoustic_measures(wav_path, t_min, t_max):
    """Returns (Intensity, F0_Reliable_Frames, Total_Frames)"""
    try:
        if not os.path.exists(wav_path): return np.nan, 0, 0
        sound = parselmouth.Sound(wav_path)
        part = sound.extract_part(from_time=t_min, to_time=t_max, preserve_times=True)

        # Intensity
        intensity = call(part.to_intensity(), "Get mean", t_min, t_max, "energy")

        # F0 Availability
        pitch = part.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=300)
        f0_vals = pitch.selected_array['frequency']
        n_total = len(f0_vals)
        n_reliable = np.sum((f0_vals > 75) & (f0_vals < 300))

        return intensity, n_reliable, n_total
    except: return np.nan, 0, 0

def calculate_local_speech_rate(wdf, target_onset, window=2.0):
    start, end = target_onset - window, target_onset + window
    count = wdf[(wdf['onset'] >= start) & (wdf['onset'] <= end)].shape[0]
    return count / (2 * window)

# 3. Load Frequency
print("🔵 Phase 0: Frequency Data...")
freq_dict = {}
try:
    r = requests.get("https://www.ugent.be/pp/experimentele-psychologie/en/research/documents/subtlexus/subtlexus2.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    with z.open(next(f for f in z.namelist() if 'SUBTLEX' in f and f.endswith('.txt'))) as f:
        df_freq = pd.read_csv(f, sep='\t')
        freq_dict = df_freq.set_index(df_freq['Word'].astype(str).str.lower())['Lg10WF'].to_dict()
except: print("⚠️ Using dummy frequency.")

# 4. Main Processing
print("🔵 Phase 1: Corpus Scanning & Pairing...")
candidates = []
FILLERS = {"uh", "um", "mm", "hm", "ah", "eh"}
txt_files = sorted(glob.glob(os.path.join(RAW_DIR, "*.txt")))

for txt_path in tqdm(txt_files, desc="Scanning"):
    base = os.path.splitext(os.path.basename(txt_path))[0]
    speaker = base[:3]
    with open(txt_path, 'r', encoding='utf-8', errors='ignore') as f: lines = f.readlines()

    tokens = []
    wc = {}
    for line in lines:
        l_toks = line.split()
        valid_idxs = [i for i, t in enumerate(l_toks) if normalize_token(t)]
        for idx, t in enumerate(l_toks):
            norm = normalize_token(t)
            nth, is_final = 0, 0
            if norm:
                wc[norm] = wc.get(norm, 0) + 1
                nth = wc[norm]
                if valid_idxs and idx == valid_idxs[-1]: is_final = 1
            tokens.append({"orig": t, "norm": norm, "nth": nth, "is_final": is_final})

    cutoff_re = re.compile(r"<CUTOFF-[^>]+>")
    for i, tok in enumerate(tokens):
        if cutoff_re.match(tok['orig']) and i > 0:
            target = tokens[i-1]['norm']
            if not target: continue
            for j in range(i+1, min(i+6, len(tokens))):
                mid = tokens[i+1:j]
                if any(m['orig'].startswith('<HES') or m['norm'] in FILLERS for m in mid): break
                if tokens[j]['norm'] == target:
                    candidates.append({
                        "basename": base, "speaker": speaker, "word": target,
                        "first_nth": tokens[i-1]['nth'], "repeat_nth": tokens[j]['nth'],
                        "first_pos": tokens[i-1]['is_final'], "repeat_pos": tokens[j]['is_final']
                    })
                    break

# Pairing & Acoustics
final_pairs = []
cand_df = pd.DataFrame(candidates)

for base, group in tqdm(cand_df.groupby("basename"), desc="Pairing & Acoustics"):
    wdf = load_words_robust(os.path.join(RAW_DIR, f"{base}.words"))
    if wdf.empty: continue

    for _, row in group.iterrows():
        f_rows = wdf[(wdf['word']==row['word']) & (wdf['nth']==row['first_nth'])]
        r_rows = wdf[(wdf['word']==row['word']) & (wdf['nth']==row['repeat_nth'])]

        if not f_rows.empty and not r_rows.empty:
            f, r = f_rows.iloc[0], r_rows.iloc[0]

            # Acoustics
            int_f, f0_k_f, f0_n_f = get_acoustic_measures(os.path.join(RAW_DIR, f"{base}.wav"), f['onset'], f['offset'])
            int_r, f0_k_r, f0_n_r = get_acoustic_measures(os.path.join(RAW_DIR, f"{base}.wav"), r['onset'], r['offset'])

            common = {"basename": base, "speaker": row['speaker'], "word": row['word'], "log_freq": freq_dict.get(row['word'], 0.0)}

            # First
            final_pairs.append({**common, "condition": "first", "duration": f['duration'], "intensity": int_f,
                                "f0_k": f0_k_f, "f0_n": f0_n_f, "utt_pos": row['first_pos'],
                                "speech_rate": calculate_local_speech_rate(wdf, f['onset'])})
            # Repeat
            final_pairs.append({**common, "condition": "repeat", "duration": r['duration'], "intensity": int_r,
                                "f0_k": f0_k_r, "f0_n": f0_n_r, "utt_pos": row['repeat_pos'],
                                "speech_rate": calculate_local_speech_rate(wdf, r['onset'])})

pairs_df = pd.DataFrame(final_pairs)

if not pairs_df.empty:
    # Cleaning
    pairs_df = pairs_df[(pairs_df['duration'] > 0.03) & (pairs_df['duration'] < 2.0)].dropna()
    pairs_df['log_duration'] = np.log(pairs_df['duration']) # Seconds basis
    pairs_df['intensity_centered'] = pairs_df['intensity'] - pairs_df.groupby('speaker')['intensity'].transform('mean')
    pairs_df['f0_avail'] = pairs_df.apply(lambda x: x['f0_k']/x['f0_n'] if x['f0_n']>0 else 0, axis=1)

    # Save for R
    pairs_df.to_csv(os.path.join(OUT_DIR, "data_for_r.csv"), index=False)

    # ---------------------------------------------------------
    # 📊 Outputs: Table 1 & Figures
    # ---------------------------------------------------------
    print("\n🔵 Generating Outputs...")
    sns.set(style="whitegrid", font_scale=1.2)
    plt.rcParams.update({'text.color':'black', 'axes.labelcolor':'black', 'xtick.color':'black', 'ytick.color':'black'})

    # Table 1
    table1 = pairs_df.groupby('condition')[['duration', 'log_duration', 'intensity_centered', 'f0_avail']].agg(['mean', 'std', 'count']).T
    table1.to_csv(os.path.join(OUT_DIR, "Table1_Descriptive_Stats.csv"))
    print(f"✅ Table 1 saved.")

    # Figure 1: Distribution
    plt.figure(figsize=(8, 6))
    sns.kdeplot(data=pairs_df, x="log_duration", hue="condition", fill=True, palette=['gray', 'black'], alpha=0.4)
    plt.title("Figure 1: Distribution of Log Duration", fontweight='bold')
    plt.savefig(os.path.join(OUT_DIR, "Figure1_Distribution.png"), dpi=300)
    plt.close()

    # Figure 2: Comparison
    plt.figure(figsize=(6, 6))
    sns.pointplot(data=pairs_df, x="condition", y="log_duration", order=["first", "repeat"], capsize=.1, color="black", markers="s")
    sns.stripplot(data=pairs_df, x="condition", y="log_duration", order=["first", "repeat"], color="gray", alpha=0.1, jitter=True)
    plt.title("Figure 2: Mean Log Duration", fontweight='bold')
    plt.savefig(os.path.join(OUT_DIR, "Figure2_Comparison.png"), dpi=300)
    plt.close()

    # Figure S1: F0 Availability
    plt.figure(figsize=(6, 6))
    sns.boxplot(data=pairs_df, x="condition", y="f0_avail", order=["first", "repeat"], showfliers=False, color="white", linecolor="black")
    sns.stripplot(data=pairs_df, x="condition", y="f0_avail", order=["first", "repeat"], color="gray", alpha=0.1, jitter=True)
    plt.title("Figure S1: F0 Availability", fontweight='bold')
    plt.savefig(os.path.join(OUT_DIR, "FigureS1_F0_Availability.png"), dpi=300)
    plt.close()

    print(f"✅ All Figures (1, 2, S1) saved to {OUT_DIR}")
    print("👉 Proceed to Cell 2 for Statistical Modeling.")

else:
    print("❌ No valid pairs found.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 112.3 MB/s eta 0:00:00
Mounted at /content/drive
🔵 Phase 0: Frequency Data...
🔵 Phase 1: Corpus Scanning & Pairing...


Pairing & Acoustics: 100%|██████████| 187/187 [02:29<00:00,  1.25it/s]



🔵 Generating Outputs...
✅ Table 1 saved.
✅ All Figures (1, 2, S1) saved to /content/drive/MyDrive/buckeye_self_repair/output
👉 Proceed to Cell 2 for Statistical Modeling.


In [ ]:
%%R
# =============================================================================
# Cell 2: R Statistical Modeling (Saves Results to Files)
# =============================================================================

# Install/Load
if (!require("lme4")) install.packages("lme4", quiet=TRUE)
if (!require("lmerTest")) install.packages("lmerTest", quiet=TRUE)
if (!require("MuMIn")) install.packages("MuMIn", quiet=TRUE)
library(lme4); library(lmerTest); library(MuMIn)

# Load Data
out_dir <- "/content/drive/MyDrive/buckeye_self_repair/output"
data <- read.csv(file.path(out_dir, "data_for_r.csv"))

# Factorize
data$condition <- as.factor(data$condition)
data$speaker <- as.factor(data$speaker)
data$word <- as.factor(data$word)
data$utt_pos <- as.factor(data$utt_pos)

# ---------------------------------------------------------
# 1. Duration Model (Table 2)
# ---------------------------------------------------------
sink(file.path(out_dir, "Table2_Duration_Model.txt"))
cat("=== TABLE 2: DURATION MODEL ===\n")
m_dur <- lmer(log_duration ~ condition + log_freq + speech_rate + utt_pos + (1|speaker) + (1|word), data=data, REML=TRUE)
print(summary(m_dur))
print(r.squaredGLMM(m_dur))
sink()

# ---------------------------------------------------------
# 2. Intensity Model (Table 3)
# ---------------------------------------------------------
sink(file.path(out_dir, "Table3_Intensity_Model.txt"))
cat("=== TABLE 3: INTENSITY MODEL ===\n")
m_int <- lmer(intensity_centered ~ condition + log_freq + speech_rate + utt_pos + (1|speaker) + (1|word), data=data, REML=TRUE)
print(summary(m_int))
print(r.squaredGLMM(m_int))
sink()

# ---------------------------------------------------------
# 3. F0 Availability Model (Supplementary)
# ---------------------------------------------------------
data$f0_fail <- data$f0_n - data$f0_k
sink(file.path(out_dir, "Supplementary_F0_Model.txt"))
cat("=== SUPPLEMENTARY: F0 MODEL ===\n")
m_f0 <- glmer(cbind(f0_k, f0_fail) ~ condition + log_freq + speech_rate + utt_pos + (1|speaker), data=data, family=binomial, control=glmerControl(optimizer="bobyqa"))
print(summary(m_f0))
sink()

cat("✅ All Statistical Tables saved to:", out_dir, "\n")

✅ All Statistical Tables saved to: /content/drive/MyDrive/buckeye_self_repair/output 


Loading required package: lme4
also installing the dependencies ‘rbibutils’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘Rdpack’, ‘RcppEigen’

Loading required package: lmerTest
also installing the dependency ‘numDeriv’

Loading required package: MuMIn
also installing the dependency ‘insight’

Loading required package: Matrix

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step

boundary (singular) fit: see help('isSingular')
In addition: Warning messages:
1: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
  there is no package called ‘lme4’
2: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
  there is no package called ‘lmerTest’
3: In library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE,  :
  there is no package called ‘MuMIn’
4: Model failed to converge with 1 negative eigenvalue: -3

In [ ]:
%%R
# =============================================================================
# Cell 5: Interaction Analysis via Likelihood Ratio Tests (LRT)
# =============================================================================

# Libraries
library(lme4)
library(lmerTest)

# Load Data
out_dir <- "/content/drive/MyDrive/buckeye_self_repair/output"
data <- read.csv(file.path(out_dir, "data_for_r.csv"))

# Factorize
data$condition <- as.factor(data$condition)
data$speaker <- as.factor(data$speaker)
data$word <- as.factor(data$word)
data$utt_pos <- as.factor(data$utt_pos)

# -----------------------------------------------------------------------------
# 1. DURATION MODEL INTERACTIONS (log_duration)
# -----------------------------------------------------------------------------
cat("\n🔵 --- LRT: DURATION INTERACTIONS ---\n")

# Base Model (ML for comparison)
m0_dur <- lmer(log_duration ~ condition + log_freq + speech_rate + utt_pos + (1|speaker) + (1|word),
               data=data, REML=FALSE)

# Interaction 1: Condition x Speech Rate
m1_dur <- lmer(log_duration ~ condition * speech_rate + log_freq + utt_pos + (1|speaker) + (1|word),
               data=data, REML=FALSE)

# Interaction 2: Condition x Utterance Position
m2_dur <- lmer(log_duration ~ condition * utt_pos + log_freq + speech_rate + (1|speaker) + (1|word),
               data=data, REML=FALSE)

cat("\n>> Test 1: Condition x Speech Rate (Duration)\n")
print(anova(m0_dur, m1_dur))

cat("\n>> Test 2: Condition x Utt Position (Duration)\n")
print(anova(m0_dur, m2_dur))


# -----------------------------------------------------------------------------
# 2. INTENSITY MODEL INTERACTIONS (intensity_centered)
# -----------------------------------------------------------------------------
cat("\n\n🔵 --- LRT: INTENSITY INTERACTIONS ---\n")

# Base Model (ML)
m0_int <- lmer(intensity_centered ~ condition + log_freq + speech_rate + utt_pos + (1|speaker) + (1|word),
               data=data, REML=FALSE)

# Interaction 1: Condition x Speech Rate
m1_int <- lmer(intensity_centered ~ condition * speech_rate + log_freq + utt_pos + (1|speaker) + (1|word),
               data=data, REML=FALSE)

# Interaction 2: Condition x Utterance Position
m2_int <- lmer(intensity_centered ~ condition * utt_pos + log_freq + speech_rate + (1|speaker) + (1|word),
               data=data, REML=FALSE)

cat("\n>> Test 3: Condition x Speech Rate (Intensity)\n")
print(anova(m0_int, m1_int))

cat("\n>> Test 4: Condition x Utt Position (Intensity)\n")
print(anova(m0_int, m2_int))


🔵 --- LRT: DURATION INTERACTIONS ---

>> Test 1: Condition x Speech Rate (Duration)
Data: data
Models:
m0_dur: log_duration ~ condition + log_freq + speech_rate + utt_pos + (1 | speaker) + (1 | word)
m1_dur: log_duration ~ condition * speech_rate + log_freq + utt_pos + (1 | speaker) + (1 | word)
       npar    AIC    BIC  logLik -2*log(L)  Chisq Df Pr(>Chisq)
m0_dur    8 1438.7 1478.7 -711.37    1422.7                     
m1_dur    9 1440.7 1485.7 -711.36    1422.7 0.0064  1     0.9365

>> Test 2: Condition x Utt Position (Duration)
Data: data
Models:
m0_dur: log_duration ~ condition + log_freq + speech_rate + utt_pos + (1 | speaker) + (1 | word)
m2_dur: log_duration ~ condition * utt_pos + log_freq + speech_rate + (1 | speaker) + (1 | word)
       npar    AIC    BIC  logLik -2*log(L)  Chisq Df Pr(>Chisq)
m0_dur    8 1438.7 1478.7 -711.37    1422.7                     
m2_dur    9 1440.4 1485.3 -711.18    1422.4 0.3764  1     0.5396


🔵 --- LRT: INTENSITY INTERACTIONS ---

>> Test 3:

boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
boundary (singular) fit: see help('isSingular')
In addition: Warning messages:
1: Model failed to converge with 1 negative eigenvalue: -3.3e+02 
2: Model failed to converge with 1 negative eigenvalue: -3.4e+02 
3: Model failed to converge with 1 negative eigenvalue: -3.3e+02 


In [ ]:
# =============================================================================
# Supplementary Analysis: F0 < 50% Calculation (Python)
# =============================================================================
import pandas as pd

# データ読み込み（もしメモリになければ）
if 'pairs_df' not in locals():
    pairs_df = pd.read_csv("/content/drive/MyDrive/buckeye_self_repair/output/data_for_r.csv")

# F0 Availability が 0.5 未満のトークンを判定
# (f0_avail列はすでに計算済み: f0_k / f0_n)
pairs_df['lt50'] = pairs_df['f0_avail'] < 0.5

# 集計
f0_stats = pairs_df.groupby('condition')['lt50'].agg(['sum', 'mean', 'count'])
f0_stats['percent'] = f0_stats['mean'] * 100

print("📊 Tokens with F0 Availability < 50%:")
print("-" * 40)
print(f"First Production:       N = {int(f0_stats.loc['first', 'sum'])} ({f0_stats.loc['first', 'percent']:.1f}%)")
print(f"Post-Repair Repetition: N = {int(f0_stats.loc['repeat', 'sum'])} ({f0_stats.loc['repeat', 'percent']:.1f}%)")
print("-" * 40)

📊 Tokens with F0 Availability < 50%:
----------------------------------------
First Production:       N = 265 (48.4%)
Post-Repair Repetition: N = 113 (20.9%)
----------------------------------------
